# NoSQL Databases

## MongoDB

In [1]:
!pip install pymongo

In [21]:
import pymongo

#### 1 Making a Connection with MongoClient

In [2]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

#### 2 Getting a Database

In [3]:
db = client.test_database

#### 3 Getting a Collection

In [4]:
collection = db.test_collection

#### 4 Documents

In [5]:
import datetime
post = {"author": "Mike",
"text": "My first blog post!",
"tags": ["mongodb", "python", "pymongo"],
"date": datetime.datetime.utcnow()}

#### 5 Inserting a Document

In [6]:
posts = db.posts
post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('5fb65235e8c4e3175801d254')

#### 6 Listing all of the collections in our database

In [7]:
db.list_collection_names()

['posts']

#### 7 Getting a Single Document With find_one()
The most basic type of query that can be performed in MongoDB is find_one(). This method
returns a single document matching a query (or None if there are no matches). It is useful when
you know there is only one matching document, or are only interested in the first match. Here we
use find_one() to get the first document from the posts collection:

In [8]:
import pprint
pprint.pprint(posts.find_one())

{'_id': ObjectId('5fb65235e8c4e3175801d254'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 19, 11, 8, 4, 228000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


find_one() also supports querying on specific elements that the resulting document must match.
To limit our results to a document with author “Mike” we do:

In [9]:
pprint.pprint(posts.find_one({"author": "Mike"}))

{'_id': ObjectId('5fb65235e8c4e3175801d254'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 19, 11, 8, 4, 228000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


If we try with a different author, like “Eliot”, we’ll get no result

In [10]:
posts.find_one({"author": "Eliot"})

#### 8 Querying By ObjectId
We can also find a post by its _id, which in our example is an ObjectId:

In [11]:
 post_id

ObjectId('5fb65235e8c4e3175801d254')

In [12]:
pprint.pprint(posts.find_one({"_id": post_id}))

{'_id': ObjectId('5fb65235e8c4e3175801d254'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 19, 11, 8, 4, 228000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


#### 9 Bulk Inserts
In order to make querying a little more interesting, let’s insert a few more documents. In addition
to inserting a single document, we can also perform bulk insert operations, by passing a list as the
first argument to insert_many(). This will insert each document in the list, sending only a single
command to the server:

In [13]:
new_posts = [{"author": "Mike",
"text": "Another post!",
"tags": ["bulk", "insert"],
"date": datetime.datetime(2009, 11, 12, 11, 14)},
{"author": "Eliot",
"title": "MongoDB is fun",
"text": "and pretty easy too!",
"date": datetime.datetime(2009, 11, 10, 10, 45)}]

In [14]:
result = posts.insert_many(new_posts)
result.inserted_ids

[ObjectId('5fb653aae8c4e3175801d255'), ObjectId('5fb653aae8c4e3175801d256')]

#### 10 Querying for More Than One Document
To get more than a single document as the result of a query we use the find() method. find()
returns a Cursor instance, which allows us to iterate over all matching documents. For example,
we can iterate over every document in the posts collection:

In [15]:
all_posts = posts.find()
for post in all_posts:
    pprint.pprint(post)

{'_id': ObjectId('5fb65235e8c4e3175801d254'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 19, 11, 8, 4, 228000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('5fb653aae8c4e3175801d255'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}
{'_id': ObjectId('5fb653aae8c4e3175801d256'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}


Just like we did with find_one(), we can pass a document to find() to limit the returned results.
Here, we get only those documents whose author is “Mike”:

In [16]:
for post in posts.find({"author": "Mike"}):
    pprint.pprint(post)

{'_id': ObjectId('5fb65235e8c4e3175801d254'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 19, 11, 8, 4, 228000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('5fb653aae8c4e3175801d255'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


#### 11 Counting
If we just want to know how many documents match a query we can perform a count_documents()
operation instead of a full query. We can get a count of all of the documents in a collection:

In [17]:
posts.count_documents({})

3

or just of those documents that match a specific query:

In [18]:
posts.count_documents({"author": "Mike"})

2

#### 12 Range Queries
MongoDB supports many different types of advanced queries. As an example, lets perform a query
where we limit results to posts older than a certain date, but also sort the results by author. Here
we use the special “$lt” operator to do a range query, and also call sort() to sort the results by
author.

In [19]:
d = datetime.datetime(2009, 11, 12, 12)
for post in posts.find({"date": {"$lt": d}}).sort("author"):
    pprint.pprint(post)

{'_id': ObjectId('5fb653aae8c4e3175801d256'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}
{'_id': ObjectId('5fb653aae8c4e3175801d255'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


#### 13 Indexing
Adding indexes can help accelerate certain queries and can also add additional functionality to
querying and storing documents. In this example, we’ll demonstrate how to create a unique index
on a key that rejects documents whose value for that key already exists in the index.
First, we’ll need to create the index:

In [22]:
result = db.profiles.create_index([('user_id', pymongo.ASCENDING)],unique=True)
sorted(list(db.profiles.index_information()))

['_id_', 'user_id_1']

Notice that we have two indexes now: one is the index on _id that MongoDB creates automatically,
and the other is the index on user_id we just created.
Now let’s set up some user profiles:

In [23]:
user_profiles = [{'user_id': 211, 'name': 'Luke'},{'user_id': 212, 'name':'Ziltoid'}]
result = db.profiles.insert_many(user_profiles)

The index prevents us from inserting a document whose user_id is already in the collection:

In [24]:
new_profile = {'user_id': 213, 'name': 'Drew'}
duplicate_profile = {'user_id': 212, 'name': 'Tommy'}
result = db.profiles.insert_one(new_profile) # This is fine.

In [25]:
result = db.profiles.insert_one(duplicate_profile) ## Try to run this and, you should get an exception!

DuplicateKeyError: E11000 duplicate key error collection: test_database.profiles index: user_id_1 dup key: { user_id: 212 }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'user_id': 1}, 'keyValue': {'user_id': 212}, 'errmsg': 'E11000 duplicate key error collection: test_database.profiles index: user_id_1 dup key: { user_id: 212 }'}

## Neo4j

In [26]:
!pip install neo4j

  Created wheel for neo4j: filename=neo4j-4.2.0-py3-none-any.whl size=89361 sha256=ab847572592831a0afa027d8bc3c09c490e946f80f01300a640c397cd3815bb7
  Stored in directory: c:\users\kenec\appdata\local\pip\cache\wheels\89\b5\69\aae43910581a9142c391d37780e02c73d2552e81057460d356
Successfully built neo4j


#### 1 Creating Connection to the Database

In [36]:
from neo4j import GraphDatabase
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "yourPassword"))

#### 2 Creating nodes.

In [37]:
def create_relationship(tx, name, friend_name):
    tx.run("MERGE (a:Person {name: $name}) ""MERGE (a)-[:KNOWS]->(friend:Person {name: $friend_name})", name=name, friend_name=friend_name)

In [38]:
def create_example(tx, name):
    tx.run("CREATE (TheBig:Guy {name: $name})",name=name)

In [39]:
with driver.session() as session:
    session.write_transaction(create_example, "Lebron")

In [40]:
with driver.session() as session:
    session.write_transaction(create_relationship, "Alice", "Bob")
    session.write_transaction(create_relationship, "Alice", "Carl")
    session.write_transaction(create_relationship, "Arthur", "Guinevere")
    session.write_transaction(create_relationship, "Arthur", "Lancelot")
    session.write_transaction(create_relationship, "Arthur", "Merlin")

#### 3 Finding nodes

In [43]:
def get_friends(tx, name):
    for record in tx.run("MATCH (a:Person)-[:KNOWS]->(friend) WHERE a.name = $name " "RETURN friend.name ORDER BY friend.name", name=name):
        print(record["friend.name"])

In [44]:
with driver.session() as session:
    session.read_transaction(get_friends, "Arthur")

Guinevere
Lancelot
Merlin


#### Close Neo4j Connection

In [45]:
driver.close()